In [1]:

import numpy as np
import pandas as pd
from validphys.api import API

In [2]:
fitname = "250218_mtop_alphas_variation_test"

In [3]:
fit = API.fit(fit=fitname)

In [4]:
# We have to know the name of the alphas point prescription (alphas_pp) to
# extract the theoryids. We have to know alphas_pp_id to identify the .csv file
# corresponding to the alphas covmat used in the fit
pps = fit.as_input()["theorycovmatconfig"]["point_prescriptions"]
for i, pp in enumerate(pps):
    if "mtop" in pp:
        mtop_pp_id = i
        mtop_pp = pp 
    elif "alphas" in pp:
        alphas_pp_id = i
        alphas_pp = pp
    else:
        continue

In [6]:

common_dict = dict(
    dataset_inputs={"from_": "fit"},
    fit=fit.name,
    fits=[fit.name],
    use_cuts="fromfit",
    metadata_group="nnpdf31_process",
)

theoryids_dict_mtop = ({
        "point_prescription": mtop_pp,
        "theoryid": {"from_": "theory"},
        "theory": {"from_": "fit"},
        "theorycovmatconfig": {"from_": "fit"},
    } | common_dict)

theoryids_dict_alphas= ({
        "point_prescription": alphas_pp,
        "theoryid": {"from_": "theory"},
        "theory": {"from_": "fit"},
        "theorycovmatconfig": {"from_": "fit"},
    } | common_dict)

In [7]:
theoryids_mtop = API.theoryids(**theoryids_dict_mtop)
theory_plus_mtop = theoryids_mtop[2].id
theory_mid_mtop = theoryids_mtop[0].id
theory_min_mtop = theoryids_mtop[1].id

theoryids_alphas = API.theoryids(**theoryids_dict_alphas)
theory_plus_alphas= theoryids_alphas[1].id
theory_mid_alphas = theoryids_alphas[0].id
theory_min_alphas = theoryids_alphas[2].id

In [8]:

thcov_input_pdf = fit.as_input()["theorycovmatconfig"]["pdf"]

# Inputs for central theory (used to construct the mtop, alphas covmat). theory_mid_mtop contains the central value for mtop and alphas.
inps_central = dict(theoryid=theory_mid_mtop, pdf=thcov_input_pdf, **common_dict)

# Inputs for plus theory (used to construct the mtop covmat)
inps_plus = [dict(theoryid=theory_plus_mtop, pdf=thcov_input_pdf, **common_dict), dict(theoryid=theory_plus_alphas, pdf=thcov_input_pdf, **common_dict)]

# Inputs for minus theory prediction (used to construct the mtop covmat)
inps_minus = [dict(theoryid=theory_min_mtop, pdf=thcov_input_pdf, **common_dict), dict(theoryid=theory_min_alphas, pdf=thcov_input_pdf, **common_dict)]

# inputs for the computation of the prediction of the fit with cov=C+S, where S
# is computed using the inps_central, inps_plus, and inps_minus dictionaries
inps_central_fit = dict(theoryid=theory_mid_mtop, pdf={"from_": "fit"}, **common_dict)

In [23]:
if fit.as_input()["resample_negative_pseudodata"] != False:
    print("The TCM assumes Gaussianity of the pseudodata, to ensure this set")
    print("resample_negative_pseudodata: False")
    print("in the n3fit runcard!")

KeyError: 'resample_negative_pseudodata'

In [9]:
prior_theorypreds_central = API.group_result_central_table_no_table(**inps_central)["theory_central"].to_frame()
prior_theorypreds_plus = pd.concat([API.group_result_central_table_no_table(**inp_plus)["theory_central"] for inp_plus in inps_plus], axis=1)
prior_theorypreds_minus = pd.concat([API.group_result_central_table_no_table(**inp_minus)["theory_central"] for inp_minus in inps_minus], axis=1)

LHAPDF 6.5.4 loading /Users/jaco/miniconda3/envs/nnpdf_dev/share/LHAPDF/240701-02-rs-nnpdf40-baseline/240701-02-rs-nnpdf40-baseline_0000.dat
240701-02-rs-nnpdf40-baseline PDF set, member #0, version 1


In [10]:
# Get the values of mtop...
mtop_plus = API.theory_info_table(theory_db_id=theory_plus_mtop).loc["mt"].iloc[0]
mtop_central = API.theory_info_table(theory_db_id=theory_mid_mtop).loc["mt"].iloc[0]
mtop_min = API.theory_info_table(theory_db_id=theory_min_mtop).loc["mt"].iloc[0]

alphas_plus = API.theory_info_table(theory_db_id=theory_plus_alphas).loc["alphas"].iloc[0]
alphas_central = API.theory_info_table(theory_db_id=theory_mid_mtop).loc["alphas"].iloc[0]
alphas_min = API.theory_info_table(theory_db_id=theory_min_alphas).loc["alphas"].iloc[0]

# ... and make sure the shift in both directions is symmetric
delta_plus = np.array([mtop_plus - mtop_central, alphas_plus - alphas_central])
delta_min = np.array([mtop_central - mtop_min, alphas_central - alphas_min]) 
if np.any(abs(delta_min - delta_plus) > 1e-6):
    raise ValueError("mtop shifts in both directions is not symmetric")
else:
    step_size = np.array(delta_min).reshape(-1, 1)

In [12]:
# At some point we scaled the covmat to account for higher order derivatives or
# to test depencence of the prior. It is not used in the final result
covmat_scaling_factor = 1 #fit.as_input().get("theorycovmatconfig",{}).get("rescale_alphas_covmat",1.0)

beta_tilde = np.sqrt(covmat_scaling_factor) * (step_size / np.sqrt(2)) * np.array([[1, -1, 0, 0], [0, 0, 1, -1]])
S_tilde = beta_tilde @ beta_tilde.T

In [13]:
S_tilde

array([[6.25e+00, 0.00e+00],
       [0.00e+00, 4.00e-06]])

In [52]:
delta_plus = (np.sqrt(covmat_scaling_factor) / np.sqrt(2)) * (
    prior_theorypreds_plus - prior_theorypreds_central
)
delta_minus = (np.sqrt(covmat_scaling_factor) / np.sqrt(2)) * (
    prior_theorypreds_minus - prior_theorypreds_central
)
# beta = np.array([delta_plus, delta_minus]).reshape(-1, 4)
# S_hat = beta_tilde @ beta.T

In [54]:
beta = np.array([delta_plus.iloc[:, 0].values, delta_minus.iloc[:, 0].values, delta_plus.iloc[:, 1].values, delta_minus.iloc[:, 1].values]).T
S_hat = beta_tilde @ beta.T

In [55]:
S = delta_plus @ delta_plus.T + delta_minus @ delta_minus.T
S = pd.DataFrame(S, index=delta_minus.index, columns=delta_minus.index)

In [56]:
stored_alphas_covmat = pd.read_csv(
    fit.path / f"tables/datacuts_theory_theorycovmatconfig_point_prescriptions{alphas_pp_id}_theory_covmat_custom_per_prescription.csv",
    index_col=[0, 1, 2],
    header=[0, 1, 2],
    sep="\t|,",
    encoding="utf-8",
    engine="python",
).fillna(0)

stored_mtop_covmat = pd.read_csv(
    fit.path / f"tables/datacuts_theory_theorycovmatconfig_point_prescriptions{mtop_pp_id}_theory_covmat_custom_per_prescription.csv",
    index_col=[0, 1, 2],
    header=[0, 1, 2],
    sep="\t|,",
    encoding="utf-8",
    engine="python",
).fillna(0)

stored_covmat = stored_alphas_covmat + stored_mtop_covmat


In [58]:
storedcovmat_index = pd.MultiIndex.from_tuples(
    [(aa, bb, np.int64(cc)) for aa, bb, cc in stored_covmat.index],
    names=["group", "dataset", "id"],
)  

# make sure theoryID is an integer, same as in S
stored_covmat = pd.DataFrame(
    stored_covmat.values, index=storedcovmat_index, columns=storedcovmat_index
)
stored_covmat = stored_covmat.reindex(S.index).T.reindex(S.index)

if not np.allclose(S, stored_covmat):
    print("Reconstructed theory covmat, S, is not the same as the stored covmat!")

In [59]:
theorypreds_fit = API.group_result_table_no_table(**inps_central_fit).iloc[:, 2:]

In [60]:
exp_covmat = API.groups_covmat(
    use_t0=True,
    datacuts={"from_": "fit"},
    t0pdfset={"from_": "datacuts"},
    theoryid= {"from_": "theory"},
    theory={"from_": "fit"},
    **common_dict
)

In [61]:
total_th_covmat = pd.read_csv(
    fit.path / f"tables/datacuts_theory_theorycovmatconfig_theory_covmat_custom.csv",
    index_col=[0, 1, 2],
    header=[0, 1, 2],
    sep="\t|,",
    encoding="utf-8",
    engine="python",
).fillna(0)
storedcovmat_index = pd.MultiIndex.from_tuples(
    [(aa, bb, np.int64(cc)) for aa, bb, cc in total_th_covmat.index],
    names=["group", "dataset", "id"],
)  # make sure theoryID is an integer, same as in S
total_th_covmat = pd.DataFrame(
    total_th_covmat.values, index=storedcovmat_index, columns=storedcovmat_index
)
total_th_covmat = total_th_covmat.reindex(S.index).T.reindex(S.index)

In [62]:
# Note that mean_prediction is different from the prediction of the mean PDF (i.e. replica0)
mean_prediction = theorypreds_fit.mean(axis=1)

X = np.zeros_like(exp_covmat.values)
for i in range(theorypreds_fit.shape[1]):
    X += np.outer(
        (theorypreds_fit.iloc[:, i] - mean_prediction),
        (theorypreds_fit.iloc[:, i] - mean_prediction),
    )
X *= 1 / theorypreds_fit.shape[1]

In [63]:
# In the computation we use <D>_rep and not the central value of the data D_exp, though if
# resample_negative_pseudodata: false
# is set in the n3fit runcard, D_exp and <D>_rep should be the same as N_rep -> inf.
pseudodata = API.read_pdf_pseudodata(**common_dict)
dat_reps = pd.concat(
    [i.pseudodata.reindex(prior_theorypreds_central.index) for i in pseudodata], axis=1
)

In [64]:
invcov = np.linalg.inv(exp_covmat + total_th_covmat)

# delta_T_tilde is Eq. 3.37 in https://arxiv.org/pdf/2105.05114
delta_T_tilde = -S_hat @ invcov @ (mean_prediction - dat_reps.mean(axis=1))

In [65]:
delta_T_tilde

array([2.55688422e+00, 2.16971678e-03])

In [ ]:

# P_tilde is Eq. 3.38.
#
# Note that not all terms of the equation in the paper are here, in particular
# X_tile and X_hat vanish. This is because they measure the covariance of
# T_tilde over PDF replicas, but for us T_tilde is alphas. The prediciton of
# alphas does not depend on the PDF, and as such T_tilde^(r) == T_tilde^(0)
P_tilde = S_hat @ invcov @ X @ invcov @ S_hat.T + S_tilde - S_hat @ invcov @ S_hat.T
central_theory = np.array([mtop_central, alphas_central])
pred = central_theory + delta_T_tilde

# print(rf"Prediction for $\m_{{top}}$: {pred:.4f} ± {P_tilde:.4f}")

In [ ]:
# next: include alphas, MHOU

In [69]:
pred

array([1.75056884e+02, 1.20169717e-01])

In [67]:
P_tilde

array([[3.82828963e-01, 1.05679336e-04],
       [1.05679612e-04, 1.46520944e-07]])

In [70]:
np.sqrt(np.diag(P_tilde))

array([6.18731738e-01, 3.82780543e-04])